## Running Julia in command window
```
C:\Users\wcp27\AppData\Local\Programs\Julia-1.6.7\bin\julia.exe <script>
```

In [1]:
# Load julia packages
using Pkg
Pkg.add("Distributions")
Pkg.add("ProgressMeter")
Pkg.add("Gen")
Pkg.add("Plots")
Pkg.add("Parameters")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("PyCall")
Pkg.add("Conda")
using Distributions
using ProgressMeter
using Gen, Plots
using DataFrames
using CSV
using PyCall
using Conda

# Download python packages
Conda.add("numpy")
Conda.pip_interop(true)
Conda.pip("install", "mitsuba")

# Import python packages/scripts
mi = pyimport("mitsuba")
np = pyimport("numpy")
@pyinclude("gen_utils.py")

# Set rendering mode
mi.set_variant("cuda_ad_rgb")
;

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes

Retrieving notices: ...working... done

WARNING conda.models.version:get_matcher(546): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.7.1.*, but conda is ignoring the .* and treating it as 1.7.1


done
Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0


[ Info: Running `conda config --set pip_interop_enabled true --file /home/wcp27/.julia/conda/3/x86_64/condarc-julia.yml` in root environment
[ Info: Running `pip install mitsuba` in root environment


In [14]:
scene = py"load_scene"("bedroom", "cat")::PyObject
#scene = py"transform_scene"(scene, "baby", 1.3, 1.3, 0)::PyObject
image = @pycall mi.render(scene, spp=256)::PyObject
bitmap = @pycall mi.Bitmap(image).convert(srgb_gamma=true)::PyObject
bitmap

PyObject Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [512, 512],
  srgb_gamma = 1,
  struct = Struct<12>[
    float32 R; // @0, gamma, premultiplied alpha
    float32 G; // @4, gamma, premultiplied alpha
    float32 B; // @8, gamma, premultiplied alpha
  ],
  data = [ 3 MiB of image data ]
]

In [3]:
@gen function room()
    # Choose scene
    scenes = ["living-room", "bedroom"]
    scene ~ Gen.categorical([0.5, 0.5])
    sscene = scenes[scene]
    
    # Choose agent
    agents = ["none", "cat", "dog", "human"]
    agent ~ Gen.categorical([0.25, 0.25, 0.25, 0.25])
    sagent = agents[agent]

    # Draw position of agent (x, y)
    if sagent == "none"
        # Draw random
        x ~ Gen.uniform(0, 0)
        z ~ Gen.uniform(0, 0)
        x_scale = x
        z_scale = z
        y_scale = 0
    else
        # Draw random
        x ~ Gen.uniform(0, 1)
        z ~ Gen.uniform(0, 1)

        # Scale
        if sscene == "living-room"
            x_scale = x*(1.3+1.75)-1.75
            z_scale = z*(1.3+0.75)-0.75
            y_scale = 0
        elseif sscene == "bedroom"
            x_scale = x*(1.5+1)-1
            z_scale = z*(1.5+0.5)-0.5

            if sagent!="human" && x_scale<0.5 && x_scale>-1 && z_scale<0.2 && z_scale>-0.5
                y_scale = 0.4
            else
                y_scale = 0
            end
        end
    end
    
    # Instantiate Scene
    scene = py"load_scene"(sscene, sagent)::PyObject
    
    # Translate agent
    scene = py"transform_scene"(scene, sagent, x_scale, z_scale, y_scale)::PyObject

    # Scene to Float Array
    mu = py"scene_to_float"(scene, 512)::Array{Float32, 3}

    # define the likelihood as a gaussian with diagonal covariance
    pred ~ broadcasted_normal(mu, 0.05)
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[], false, Union{Nothing, Some{Any}}[], var"##room#275", Bool[], false)

In [2]:
@gen function room()
    # Choose scene
    scenes = ["living-room", "bedroom"]
    scene ~ Gen.categorical([0.5, 0.5])
    sscene = scenes[scene]
    
    # Choose agent
    agents = ["none", "cat", "dog", "human"]
    agent ~ Gen.categorical([0.25, 0.25, 0.25, 0.25])
    sagent = agents[agent]

    # Draw position of agent (x, y)
    if sagent == "none"
        # Draw random
        x ~ Gen.uniform(0, 0)
        z ~ Gen.uniform(0, 0)
        x_scale = x
        z_scale = z
        y_scale = 0
    else
        # Draw random
        x ~ Gen.uniform(0, 1)
        z ~ Gen.uniform(0, 1)

        # Scale
        if sscene == "living-room"
            x_scale = x*(1.3+1.75)-1.75
            z_scale = z*(1.3+0.75)-0.75
            y_scale = 0
        elseif sscene == "bedroom"
            x_scale = x*(1.5+1)-1
            z_scale = z*(1.5+0.5)-0.5

            if sagent!="human" && x_scale<=0.5 && x_scale>=-1 && z_scale<=0.2 && z_scale>=-0.5
                y_scale = 0.4
            else
                y_scale = 0
            end
        end
    end
    
    # Instantiate Scene
    scene = py"load_scene"(sscene, sagent)::PyObject
    
    # Translate agent
    scene = py"transform_scene"(scene, sagent, x_scale, z_scale, y_scale)::PyObject

    # Scene to Float Array
    mu = py"scene_to_float"(scene, 512)::Array{Float32, 3}

    # Blurr image Float
    mu = py"blurr_float_img"(mu, 4)::Array{Float32, 3}

    # define the likelihood as a gaussian with diagonal covariance
    pred ~ broadcasted_normal(mu, 0.05)
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[], false, Union{Nothing, Some{Any}}[], var"##room#275", Bool[], false)

In [ ]:
tr = Gen.simulate(room, ())

In [4]:
# use the generative model to generate synthetic data
function data_maker()
    df = DataFrame(image=String[], living_room=Int[], bedroom=Int[], none=Int[], cat=Int[], dog=Int[], human=Int[], x=Float32[], z=Float32[])

    @showprogress for k = 1:2000
        # simulate data using the prior 
        tr = Gen.simulate(room, ())
        
        # Extract information
        scenes = ["living-room", "bedroom"]
        scene = tr[:scene]
        sscene = scenes[scene]

        vscene = zeros(Int8, length(scenes))
        vscene[scene] = 1
            
        agents = ["none", "cat", "dog", "human"]
        agent = tr[:agent]
        sagent = agents[agent]

        vagent = zeros(Int8, length(agents))
        vagent[agent] = 1
        
        if sagent == "none"
            # Draw random
            x = tr[:x]
            z = tr[:z]
            x_scale = x
            z_scale = z
        else
            # Draw random
            x = tr[:x]
            z = tr[:z]

            # Scale
            if sscene == "living-room"
                x_scale = x*(1.3 + 1.75)-1.75
                z_scale = z*(1.3 + 0.75)-0.75
            elseif sscene == "bedroom"
                x_scale = x*(1+0.9)-0.9
                z_scale = z*(1+0.3)-0.3  
            end
        end

        # Save room as png
        # Pull prediction scene
        pred = tr[:pred]

        # Save pred
        filename = join([sscene, sagent, string(k)*".png"], '_')
        py"array_png"(pred, "training", filename)

        # Save to dataframe
        push!(df, [filename, vscene[1], vscene[2], vagent[1], vagent[2], vagent[3], vagent[4], x, z])
    end
    CSV.write("./images/training/labels.csv", df)

    return nothing
end
data_maker()

Progress: 100%|█████████████████████████████████████████| Time: 2:17:31
